In [32]:
import tensorflow as tf
import pandas as pd
import numpy as np
from src.metric import amex_metric
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
train = pd.read_parquet("../data/processed/train_w_labels.parquet")

In [21]:
train = train.reset_index()

In [33]:
model = keras.Sequential([
    layers.Dense(256, activation="relu"),
    layers.Dense(256, activation = "relu"),
    layers.Dense(1, activation="sigmoid")
])

In [ ]:
def amex_metric(y_true, y_pred):

    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:, 0] == 0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:, 0]) / np.sum(labels[:, 0])

    gini = [0, 0]
    for i in [1, 0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:, 0] == 0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] * weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1] / gini[0] + top_four)



In [22]:
FEATURES = train.columns.to_list()

In [18]:
target = FEATURES[-1]

In [24]:
FEATURES=FEATURES[2:]

In [25]:
target = train.target

In [27]:
train = train [FEATURES]

In [35]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=amex_metric)

In [36]:
history = model.fit(train,target, epochs=20, batch_size=512, validation_split= .3)

Epoch 1/20


AttributeError: in user code:

    File "/home/victor/Documents/1-DataScience/1-Projetos/1-Default/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_22043/2824184094.py", line 26, in amex_metric  *
        df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})

    AttributeError: Tensor object has no attribute 'ravel'. 
            If you are looking for numpy-related methods, please run the following:
            from tensorflow.python.ops.numpy_ops import np_config
            np_config.enable_numpy_behavior()
          
